In [7]:
import sys, os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../../"))
from scheduler import Scheduler, file_estimated_cost
import pandas as pd
import pickle as pkl
import heapq
import time
import multiprocessing as mp

In [14]:
with open("../queue.pkl", "rb") as fp:
	queue =	pkl.load(fp)

ExtractionSizes = pd.read_csv("../AggregateExtractionTimes/ExtractionSizes.csv", index_col=0)
ExtractionTimes = pd.read_csv("../AggregateExtractionTimes/ExtractionTimes.csv", index_col=0)

print(len(queue))

20369


In [9]:
count = 0
real = 0
empty = 0


kw = 0
tab = 0
unknown = 0
jsonxml = 0
netcdf = 0
image = 0

for i in range(len(queue)):
	file_item = queue[i]
	best_index = file_item.best_extractor_index()
	filename = file_item.get_filename()
	best_time = ExtractionTimes.loc[filename][best_index]
	if best_index == 2:
		assert best_time == 5
		count += 1

	if os.path.getsize(filename) == 0:
		empty += 1
	else:	
		if best_index == 0:
			kw += 1
		elif best_index == 1: 
			tab += 1
		elif best_index == 2:
			unknown += 1
		elif best_index == 3: 
			jsonxml += 1
		elif netcdf == 4:
			netcdf += 1
		else:
			image += 1


print("Unknown Extractor: ", count)
print("Best Extractor is empty: ", empty)
print("Keywords:", kw)
print("Tabular:", tab)
print("Unknown:", unknown)
print("JSONXML:", jsonxml)
print("NetCDF:", netcdf)
print("Image:", image)



Unknown Extractor:  73
Best Extractor is empty:  7
Keywords: 2935
Tabular: 6196
Unknown: 73
JSONXML: 3180
NetCDF: 0
Image: 7978


In [13]:
def simulate_single_file_extraction(queue, iter, lock, ExtractionTimes, ExtractionSizes):
	best_index = 2 # 2 represents unknown
	extraction_time = 5
	while len(queue) > 0: # nonzero extraction time AND not unknown
		file_item = heapq.heappop(queue)
		best_index = file_item.best_extractor_index()
		if best_index == 2 or extraction_time == 5:
			continue


		extraction_time = ExtractionTimes.loc[file_item.get_filename()][best_index]
	'''
	if ExtractionSizes[file_item.get_filename()][best_index] > 0:	
		lock.acquire()
		try:
			
			if best_index == 0:
				kw.value += 1
			elif best_index == 1: 
				tab.value += 1
			elif best_index == 2:
				unknown.value += 1
			elif best_index == 3: 
				jsonxml.value += 1
			elif netcdf == 4:
				netcdf.value += 1
			else:
				image.value += 1
		finally:
			lock.release()
	'''

	time.sleep(extraction_time)
	
	lock.acquire()
	try:
		iter.value += 1
	finally:
		lock.release()
	

In [ ]:
print(mp.cpu_count())

In [15]:
start_time = time.time()
iterations = mp.Value("i", 0)

lock = mp.Lock()
processes=[mp.Process(target=simulate_single_file_extraction, args=(queue, iterations, lock, ExtractionTimes,
                              ExtractionSizes)) for x in range(0, mp.cpu_count())]

for p in processes:
    p.start()

for p in processes:
    p.join()

print(iterations)
print("--- %s seconds ---" % (time.time() - start_time))

<Synchronized wrapper for c_int(48)>
--- 11.376144170761108 seconds ---
